In [1]:
import path_configs # noqa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import settings
settings.init()
from modules.EncoderGenerators import * # noqa

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# assumes that array is not zero
def scaled(tensor):
    return (tensor-tf.math.reduce_min(tensor))/(tf.math.reduce_max(tensor)-tf.
                                                math.reduce_min(tensor))

TRAIN_DATA_LEN: 3720334
VAL_DATA_LEN: 1593071


In [5]:
from DataPreprocessor import DataLoader
dl = DataLoader()
X_array = dl.make_spectrogram(1241)

In [6]:
X_array.shape

TensorShape([1025, 2975])

In [ ]:
class TrainClassifierGenerator(tf.data.Dataset):
    def _generator(case_nums, TIMESTEPS, WINDOWS_STEP, BATCH_SIZE):
        encoder = tf.keras.models.load_model('saved_models\encoder_model_1_1600375052_1600376295.1361377.h5')
        from DataPreprocessor import DataLoader
        dl = DataLoader()
        spectral_frames = []
        for case_num in case_nums:
            X_array = dl.make_spectrogram(case_num)  # to map
            frame_step = 64
            max_len = max(settings.LENGTHS)//frame_step - (frame_step//2) + 1
            padding = (max_len - X_array.shape[1])//2
            X_array = tf.pad(X_array, ([0,0], [padding,padding]))
            silence = np.argwhere(np.all(X_array[..., :] == -80, axis=0))            
            X_array = tf.cast(np.swapaxes(np.delete(X_array, silence, axis=1),
                                          0, 1), tf.float32)
            L = X_array.shape[0]
            batch_start = 0
            batch_end = TIMESTEPS
            while batch_start < L-TIMESTEPS:
                limit = min(batch_end, L)
                spectral_frame = X_array[batch_start:limit, :]
                spectral_frames.append(spectral_frame)
                if len(spectral_frames) >= BATCH_SIZE:
                    spectral_frames = tf.stack(spectral_frames)
                    yield(spectral_frames)
                    spectral_frames = []
                batch_start += WINDOWS_STEP
                # window sliding by half of a window size
                batch_end += WINDOWS_STEP

    def __new__(cls,
                case_nums=settings.AE_TRAIN_IDX,
                TIMESTEPS=settings.AE_TIMESTEPS,
                WINDOWS_STEP=settings.AE_WINDOWS_STEP,
                N_FEATURES=settings.AE_N_FEATURES,
                BATCH_SIZE=settings.AE_BATCH_SIZE):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=tf.dtypes.float32,
            output_shapes=(BATCH_SIZE, TIMESTEPS, N_FEATURES),
            args=(case_nums, TIMESTEPS, WINDOWS_STEP, BATCH_SIZE)
        )